This code implements a simple blackjack environment that follows basic rules. Let's break down the approach used in this code:

1. **Initialization** (`__init__` method):
   - The `BlackJack` class is initialized with attributes such as the deck of cards, player's and dealer's hands, game status flags, scores, balance, and bet.
   - The deck is represented as a list of cards, where each card is represented by its value ('2' to '10', 'J', 'Q', 'K', 'A'), and it is initialized with four copies of each card.
   - Other attributes such as `player_hand`, `dealer_hand`, `game_over`, `player_stand`, `dealer_stand`, `state`, `reward`, `player_score`, `dealer_score`, `balance`, and `bet` are initialized accordingly.

2. **Resetting the Environment** (`reset` method):
   - This method resets the game to its initial state.
   - It reshuffles the deck, deals two cards each to the player and the dealer, and sets various attributes accordingly.

3. **Placing Bets** (`place_bet` method):
   - This method allows the player to place a bet before starting the game.
   - It checks if the player has sufficient balance and deducts the bet amount from the balance if valid.

4. **Calculating Scores** (`calculate_score` method):
   - This method calculates the score of a given hand according to the rules of blackjack.
   - Face cards ('J', 'Q', 'K') count as 10, and 'Aces' can count as 1 or 11 depending on the situation.

5. **Updating Balance** (`update_balance` method):
   - This method updates the player's balance based on the game outcome (win, lose, draw).

6. **Player's Turn** (`player_turn` method):
   - This method handles the player's turn, allowing them to either 'hit' (request another card) or 'stand' (stop requesting cards).
   - It updates the player's hand and score accordingly.

7. **Dealer's Turn** (`dealer_turn` method):
   - This method handles the dealer's turn, where the dealer must hit until they reach a score of 17 or more.
   - It updates the dealer's hand and score accordingly.

8. **Taking a Step** (`step` method):
   - This method takes a step in the game based on the player's action.
   - It updates the game state and returns the new state and reward.

9. **Checking the Winner** (`check_winner` method):
   - This method determines the winner of the game based on the final scores.
   - It returns a message indicating the outcome of the game (player wins, dealer wins, draw, or bust).

10. **Main Functionality** (`main` function):
    - This function runs the main game loop.
    - It allows the player to place bets and take actions (hit or stand) until the game is over.
    - After each round, it displays the game status, including player and dealer hands, scores, rewards, and balance.
    - It checks if the player wants to cash out or if they have run out of balance to end the game.

Overall, this approach provides a simple yet comprehensive implementation of a blackjack environment where a player can interact with the game and play against a dealer. The code is well-structured and modular, making it easy to understand and extend with additional rules or functionalities.

In [ ]:
import random

class BlackJack:
    def __init__(self):
        self.deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'] * 4
        self.player_hand = []
        self.dealer_hand = []
        self.game_over = False
        self.player_stand = False
        self.dealer_stand = False
        self.state = (0, 0)
        self.reward = 0
        self.player_score = 0
        self.dealer_score = 0
        self.balance = 100
        self.bet = 0
        self.insurance_bet = 0
        self.insurance_taken = False
        self.double_down = False

    def reset(self):
        self.deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A'] * 4
        random.shuffle(self.deck)
        self.player_hand = [self.deck.pop(), self.deck.pop()]
        self.dealer_hand = [self.deck.pop(), self.deck.pop()]
        self.game_over = False
        self.player_stand = False
        self.dealer_stand = False
        self.state = (self.calculate_score(self.player_hand), self.dealer_hand[0])
        self.reward = 0
        self.player_score = 0
        self.dealer_score = 0
        self.bet = 0
        self.insurance_bet = 0
        self.insurance_taken = False
        self.double_down = False

    def place_bet(self, amount):
        if amount > self.balance:
            print("Insufficient balance to place that bet!")
            return False
        self.balance -= amount
        self.bet = amount
        return True

    def place_insurance(self):
        insurance_amount = self.bet / 2
        if insurance_amount > self.balance:
            print("Insufficient balance to place insurance bet!")
            return False
        self.balance -= insurance_amount
        self.insurance_bet = insurance_amount
        self.insurance_taken = True
        return True

    def double_down_bet(self):
        if self.bet > self.balance:
            print("Insufficient balance to double down!")
            return False
        self.balance -= self.bet
        self.bet *= 2
        self.double_down = True
        self.player_hand.append(self.deck.pop())
        self.player_stand = True
        self.player_score = self.calculate_score(self.player_hand)
        if self.player_score > 21:
            self.game_over = True
            self.reward = -1
        return True

    def calculate_score(self, hand):
        score = 0
        aces = 0
        for card in hand:
            if card in ['J', 'Q', 'K']:
                score += 10
            elif card == 'A':
                score += 11
                aces += 1
            else:
                score += int(card)
        while score > 21 and aces:
            score -= 10
            aces -= 1
        return score

    def update_balance(self):
        winner_message = self.check_winner()

        if "You went bust, dealer wins......" in winner_message:
            self.reward = -1
        elif "Dealer went bust, you win!" in winner_message or "You Win!" in winner_message:
            self.reward = 1
            self.balance += self.bet * 2
        elif "Dealer wins...." in winner_message:
            self.reward = -1
        elif "Its a Draw!" in winner_message:
            self.reward = 0
            self.balance += self.bet

        if self.insurance_taken and self.calculate_score(self.dealer_hand) == 21:
            self.balance += self.insurance_bet * 2

    def player_turn(self, action):
        if action == 1:  # hit
            self.player_hand.append(self.deck.pop())
        self.player_score = self.calculate_score(self.player_hand)
        if self.player_score > 21:
            self.game_over = True
            self.reward = -1
        elif action == 2:  # stand
            self.player_stand = True
        elif action == 3:  # double down
            self.double_down_bet()
            if self.double_down:
                self.player_turn(2)

    def dealer_turn(self):
        while self.calculate_score(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.pop())
        self.dealer_stand = True
        self.dealer_score = self.calculate_score(self.dealer_hand)
        if self.dealer_score > 21:
            self.reward = 1
        elif self.dealer_score > self.player_score:
            self.reward = -1
        elif self.dealer_score < self.player_score:
            self.reward = 1
        else:
            self.reward = 0
        self.game_over = True

    def step(self, action):
        if not self.game_over:
            if not self.player_stand:
                self.player_turn(action)
            if self.player_stand and not self.dealer_stand:
                self.dealer_turn()
        self.state = (self.calculate_score(self.player_hand), self.dealer_hand[0] if len(self.dealer_hand) > 0 else None)
        return self.state, self.reward

    def get_player_hand(self):
        return self.player_hand

    def get_dealer_hand(self):
        if not self.dealer_stand:
            return [self.dealer_hand[0], '*']
        else:
            return self.dealer_hand

    def get_player_score(self):
        return self.player_score

    def get_dealer_score(self):
        return self.dealer_score

    def get_game_over(self):
        return self.game_over

    def get_player_stand(self):
        return self.player_stand

    def get_dealer_stand(self):
        return self.dealer_stand

    def get_deck(self):
        return self.deck

    def get_state(self):
        return self.calculate_score(self.player_hand), self.dealer_hand[0] if len(self.dealer_hand) > 0 else None

    def get_actions(self):
        if self.game_over:
            return []
        actions = ['hit', 'stand']
        if len(self.player_hand) == 2:
            actions.append('double down')
        if self.dealer_hand[0] == 'A' and not self.insurance_taken:
            actions.append('insurance')
        return actions

    def check_balance(self):
        return self.balance

    def check_winner(self):
        player_score = self.calculate_score(self.player_hand)
        dealer_score = self.calculate_score(self.dealer_hand)

        if player_score > 21:
            return "You went bust, dealer wins......"
        elif dealer_score > 21:
            return "Dealer went bust, you win!"
        elif player_score > dealer_score:
            return "You Win!"
        elif dealer_score > player_score:
            return "Dealer wins...."
        else:
            return "Its a Draw!"

def main():
    env_blackjack = BlackJack()
    while True:
        env_blackjack.reset()
        print(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")
        print(f"Your balance is: ${env_blackjack.check_balance()}")
        bet = int(input("Enter your bet: "))
        if not env_blackjack.place_bet(bet):
            continue

        print("========================================================================================")
        while not env_blackjack.get_game_over():
            print(f"Your hand: {env_blackjack.get_player_hand()} (Score: {env_blackjack.get_player_score()})")
            print(f"Dealer's visible card: {env_blackjack.get_dealer_hand()[0]}")
            actions = env_blackjack.get_actions()
            print("Choose an action:")
            for i, action in enumerate(actions):
                print(f"({i + 1}) {action.capitalize()}")
            action = int(input("Enter the number of your action: ")) - 1
            selected_action = actions[action]
            if selected_action == 'insurance':
                if not env_blackjack.place_insurance():
                    continue
            state, reward = env_blackjack.step(action + 1)
            print("---------------------------------------------------------------------------------------")
            print(f'Action taken: {selected_action.capitalize()} \nCurrent state: {state} \nReward: {reward}')
            print(f"Your hand: {env_blackjack.get_player_hand()} \n(Score: {env_blackjack.get_player_score()})")
            if env_blackjack.get_dealer_stand():
                print(f"Dealer's hand: {env_blackjack.get_dealer_hand()} \n(Score: {env_blackjack.get_dealer_score()})")
            else:
                print(f"Dealer's visible card: {env_blackjack.get_dealer_hand()[0]}")

        print("**********************************************************")
        print(f"Game Over! Final Results:")
        print(f"Your final hand: {env_blackjack.get_player_hand()} (Score: {env_blackjack.get_player_score()})")
        print(f"Dealer's final hand: {env_blackjack.get_dealer_hand()} (Score: {env_blackjack.get_dealer_score()})")
        print(f"\nRound outcome: {env_blackjack.check_winner()}")
        if env_blackjack.reward != -1:
            env_blackjack.update_balance()
        print(f"Your balance is now: ${env_blackjack.check_balance()}")
        print(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")

        if env_blackjack.check_balance() <= 0:
            print("You have run out of balance. Game Over!")
            break

        print("Do you want to cash out? (1) Yes, (2) No")
        cash_out = int(input("Enter the number of your choice: "))
        if cash_out == 1:
            print(f"\nYou cashed out with a balance of ${env_blackjack.check_balance()}.")
            break
        print(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::")

if __name__ == "__main__":
    main()


:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Your balance is: $100
Enter your bet: 53
Your hand: [9, 5] (Score: 0)
Dealer's visible card: 10
Choose an action:
(1) Hit
(2) Stand
(3) Double down
Enter the number of your action: 1
---------------------------------------------------------------------------------------
Action taken: Hit 
Current state: (24, 10) 
Reward: -1
Your hand: [9, 5, 'K'] 
(Score: 24)
Dealer's visible card: 10
**********************************************************
Game Over! Final Results:
Your final hand: [9, 5, 'K'] (Score: 24)
Dealer's final hand: [10, '*'] (Score: 0)

Round outcome: You went bust, dealer wins......
Your balance is now: $47
:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Do you want to cash out? (1) Yes, (2) No
